In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Create the example vector from the chapter
vector_example = np.array([0.123, 0.321, 0.312, 0.231])

print("Our 4-dimensional vector:")
print(vector_example)
print(f"Vector shape: {vector_example.shape}")
print(f"Vector type: {type(vector_example)}")

Our 4-dimensional vector:
[0.123 0.321 0.312 0.231]
Vector shape: (4,)
Vector type: <class 'numpy.ndarray'>


In [2]:
# Create two more vectors to compare with our original
vector_similar = np.array([0.120, 0.318, 0.315, 0.228])  # Very similar to original
vector_different = np.array([0.890, 0.012, 0.654, 0.099])  # Very different

print("Original vector:", vector_example)
print("Similar vector: ", vector_similar)
print("Different vector:", vector_different)

# Calculate similarity using dot product (basic similarity measure)
similarity_1 = np.dot(vector_example, vector_similar)
similarity_2 = np.dot(vector_example, vector_different)

print(f"\nSimilarity between original and similar: {similarity_1:.4f}")
print(f"Similarity between original and different: {similarity_2:.4f}")

Original vector: [0.123 0.321 0.312 0.231]
Similar vector:  [0.12  0.318 0.315 0.228]
Different vector: [0.89  0.012 0.654 0.099]

Similarity between original and similar: 0.2678
Similarity between original and different: 0.3402


In [3]:
# Let's simulate what happens when we convert text to vectors
# In real RAG, this is done by embedding models, but we'll create simple examples

# Imagine these are documents in a company knowledge base
documents = {
    "doc1": "Python programming language tutorial",
    "doc2": "Python coding best practices guide", 
    "doc3": "Recipe for chocolate cake baking"
}

# Simulate vectors for these documents (in reality, these come from embedding models)
# Higher values in position [0] = programming-related, [1] = tutorial-related, 
# [2] = food-related, [3] = general helpfulness
doc_vectors = {
    "doc1": np.array([0.9, 0.8, 0.1, 0.7]),  # High programming, high tutorial
    "doc2": np.array([0.9, 0.6, 0.1, 0.8]),  # High programming, medium tutorial  
    "doc3": np.array([0.1, 0.3, 0.9, 0.6])   # Low programming, high food
}

# User asks: "How do I learn Python programming?"
# This gets converted to a vector too
user_query_vector = np.array([0.8, 0.9, 0.1, 0.5])  # High programming, very high tutorial

print("User Query: 'How do I learn Python programming?'")
print("Query vector:", user_query_vector)
print("\nComparing with documents:")

for doc_id, doc_vector in doc_vectors.items():
    similarity = np.dot(user_query_vector, doc_vector)
    print(f"{doc_id} ('{documents[doc_id]}'): similarity = {similarity:.2f}")

User Query: 'How do I learn Python programming?'
Query vector: [0.8 0.9 0.1 0.5]

Comparing with documents:
doc1 ('Python programming language tutorial'): similarity = 1.80
doc2 ('Python coding best practices guide'): similarity = 1.67
doc3 ('Recipe for chocolate cake baking'): similarity = 0.74


In [5]:
# STAGE 1: INDEXING - Store our documents and their vectors

class SimpleRAG:
    def __init__(self):
        self.document_store = {}  # Original documents
        self.vector_store = {}  # Document vectors

    def add_document(self, doc_id, text, vector):
        """Index a document (convert to vector and store)"""
        self.document_store[doc_id] = text
        self.vector_store[doc_id] = vector
        print(f"Indexed: {doc_id}")

# Create our RAG system and index our documents
rag_system = SimpleRAG()

# Index our example documents
for doc_id, text in documents.items():
    rag_system.add_document(doc_id, text, doc_vectors[doc_id])

print(f"\nIndexed {len(rag_system.document_store)} documents")

Indexed: doc1
Indexed: doc2
Indexed: doc3

Indexed 3 documents


In [6]:
# STAGE 2: RETRIEVAL - Find most relevant documents
def retrieve_documents(rag_system, query_vector, top_k=2):
    """Find the most similar documents to the query"""
    similarities = {}
    
    # Calculate similarity with each stored document
    for doc_id, doc_vector in rag_system.vector_store.items():
        similarity = np.dot(query_vector, doc_vector)
        similarities[doc_id] = similarity
    
    # Sort by similarity (highest first) and return top_k
    sorted_docs = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_docs[:top_k]

# Test retrieval with our Python learning query
print("RETRIEVAL STAGE:")
print("Query: 'How do I learn Python programming?'")

retrieved_docs = retrieve_documents(rag_system, user_query_vector, top_k=2)

print(f"\nTop {len(retrieved_docs)} most relevant documents:")
for doc_id, similarity in retrieved_docs:
    original_text = rag_system.document_store[doc_id]
    print(f"- {doc_id}: '{original_text}' (similarity: {similarity:.2f})")

RETRIEVAL STAGE:
Query: 'How do I learn Python programming?'

Top 2 most relevant documents:
- doc1: 'Python programming language tutorial' (similarity: 1.80)
- doc2: 'Python coding best practices guide' (similarity: 1.67)


In [7]:
# STAGE 3: GENERATION - Create response using retrieved documents
def generate_response(query, retrieved_docs, rag_system):
    """Simulate LLM generation using retrieved documents"""
    
    # Gather the retrieved document texts
    context_docs = []
    for doc_id, similarity in retrieved_docs:
        doc_text = rag_system.document_store[doc_id]
        context_docs.append(doc_text)
    
    # Simulate the prompt that would be sent to the LLM
    context = " | ".join(context_docs)
    
    simulated_prompt = f"""
    You are a helpful assistant. Answer the question using the provided context.
    
    Question: {query}
    Context: {context}
    
    Answer: Based on the context provided, here are the relevant resources for learning Python programming...
    """
    
    print("GENERATION STAGE:")
    print("=" * 50)
    print("Simulated LLM Prompt:")
    print(simulated_prompt)
    
    return context_docs

# Complete RAG pipeline test
query = "How do I learn Python programming?"
print("COMPLETE RAG PIPELINE TEST")
print("=" * 50)

# Stage 2: Retrieve
retrieved = retrieve_documents(rag_system, user_query_vector, top_k=2)

# Stage 3: Generate
context_used = generate_response(query, retrieved, rag_system)

print(f"\nPipeline completed! Used {len(context_used)} documents to generate response.")

COMPLETE RAG PIPELINE TEST
GENERATION STAGE:
Simulated LLM Prompt:

    You are a helpful assistant. Answer the question using the provided context.

    Question: How do I learn Python programming?
    Context: Python programming language tutorial | Python coding best practices guide

    Answer: Based on the context provided, here are the relevant resources for learning Python programming...
    

Pipeline completed! Used 2 documents to generate response.


In [8]:
# IMPROVED STAGE 3: Show what a real LLM response would look like
def generate_complete_response(query, retrieved_docs, rag_system):
    """Show what a complete RAG response would look like"""
    
    # Gather context
    context_docs = []
    for doc_id, similarity in retrieved_docs:
        doc_text = rag_system.document_store[doc_id]
        context_docs.append(f"- {doc_text}")
    
    # Simulate what the LLM would actually generate
    simulated_llm_response = f"""Based on the retrieved documents, here are the best resources for learning Python programming:

{chr(10).join(context_docs)}

I recommend starting with the "Python programming language tutorial" as it's specifically designed for learning, then moving to the "Python coding best practices guide" to improve your coding skills."""
    
    print("COMPLETE RAG RESPONSE:")
    print("=" * 50)
    print(f"Question: {query}")
    print()
    print("Answer:")
    print(simulated_llm_response)
    
    return simulated_llm_response

# Test the complete pipeline
print("FULL RAG PIPELINE WITH REALISTIC OUTPUT:")
print("=" * 60)

# Run all 3 stages
retrieved = retrieve_documents(rag_system, user_query_vector, top_k=2)
final_answer = generate_complete_response(query, retrieved, rag_system)

print("\n" + "=" * 60)
print("RAG PIPELINE COMPLETE!")
print("✅ Indexing: Stored documents as vectors")
print("✅ Retrieval: Found most relevant documents") 
print("✅ Generation: Created answer using retrieved context")

FULL RAG PIPELINE WITH REALISTIC OUTPUT:
COMPLETE RAG RESPONSE:
Question: How do I learn Python programming?

Answer:
Based on the retrieved documents, here are the best resources for learning Python programming:

- Python programming language tutorial
- Python coding best practices guide

I recommend starting with the "Python programming language tutorial" as it's specifically designed for learning, then moving to the "Python coding best practices guide" to improve your coding skills.

RAG PIPELINE COMPLETE!
✅ Indexing: Stored documents as vectors
✅ Retrieval: Found most relevant documents
✅ Generation: Created answer using retrieved context
